# NLP Final Project- Formality Classification

## 1. Load the Dataset

We will load the dataset and visualize the data outputs.

In [ ]:
# Importing dataset
from google.colab import drive
drive.mount('/content/drive')

# Unzip dataset
#!unzip /content/drive/MyDrive/NLP\ Project/GYAFC_Corpus.zip -d /content/drive/MyDrive/NLP\ Project/



# Load dataset
!ls "/content/drive/MyDrive/NLP Project/GYAFC_Corpus/Entertainment_Music"

Mounted at /content/drive
model_outputs  test  train  tune


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Apr 30 20:04:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
base_path = '/content/drive/MyDrive/NLP Project/GYAFC_Corpus/Entertainment_Music'
import os
# Function to read the sentences from a file
def load_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = file.readlines()
    return [s.strip() for s in sentences]  # Strip to remove any extra whitespace

# Paths to the files
formal_file_path = os.path.join(base_path, 'train', 'formal')
informal_file_path = os.path.join(base_path, 'train', 'informal')

test_formal_file_path = os.path.join(base_path, 'test', 'formal')
test_informal_file_path = os.path.join(base_path, 'test', 'informal')

# Load the sentences
formal_sentences = load_sentences(formal_file_path)
informal_sentences = load_sentences(informal_file_path)

test_formal_sentences = load_sentences(test_formal_file_path)
test_informal_sentences = load_sentences(test_informal_file_path)

In [ ]:
# Check the first 2 loaded sentences
print("First 2 Informal Sentences:", informal_sentences[:2])
print("First 2 Formal Sentences:", formal_sentences[:2])

print("First 2 Test Informal Sentences:", test_informal_sentences[:2])
print("First 2 Test Formal Sentences:", test_formal_sentences[:2])

# Count the number of sentences
num_formal_sentences = len(formal_sentences)
num_informal_sentences = len(informal_sentences)

print("Number of Formal Sentences:", num_formal_sentences)
print("Number of Informal Sentences:", num_informal_sentences)

print("Number of Test Formal Sentences:", len(test_formal_sentences))
print("Number of Test Informal Sentences:", len(test_informal_sentences))

First 2 Informal Sentences: ['the movie The In-Laws not exactly a holiday movie but funny and good!', 'that page did not give me viroses(i think)']
First 2 Formal Sentences: ["The In-Laws movie isn't a holiday movie, but it's okay.", "I don't think that page gave me viruses."]
First 2 Test Informal Sentences: ['Is Any Baby Really A Freak.', 'aspen colorado has he best music festivals, you sit all over the moutians its  on and just hang out']
First 2 Test Formal Sentences: ['I like Rhythm and Blue music.', "There's nothing he needs to change."]
Number of Formal Sentences: 52595
Number of Informal Sentences: 52595
Number of Test Formal Sentences: 1082
Number of Test Informal Sentences: 1416


## 2. Preprocessing Data

We will use **Character-based preprocessing.** This method involves converting all text to a uniform case, tokenizing at the character level, and padding sequences to a fixed length.

It has higher accuracy in the paper (https://arxiv.org/pdf/2204.08975.pdf)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Combine the datasets
all_sentences = formal_sentences + informal_sentences

test_all_sentences = test_formal_sentences + test_informal_sentences

# Initialize tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(all_sentences)

tokenizer.fit_on_texts(test_all_sentences)
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(all_sentences)
test_sequences = tokenizer.texts_to_sequences(test_all_sentences)

# Padding sequences
max_length = max([len(seq) for seq in sequences])  # Or you can define a max length
X_padded = pad_sequences(sequences, maxlen=max_length, padding='post')
test_X_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

# Prepare labels
y = [1] * len(formal_sentences) + [0] * len(informal_sentences)  # 1 for formal, 0 for informal
test_y = [1] * len(test_formal_sentences) + [0] * len(test_informal_sentences)  # 1 for formal, 0 for informal

# Print the first 2 padded sequences and labels

print("Padded Sequences:\n", X_padded[:2])  # Show first two padded sequences
print("Labels:", y[:2])  # Show first two labels

# Model Input
X = X_padded
test_X = test_X_padded

# Model Output
y = y
test_y = test_y

Padded Sequences:
 [[ 3  9  2 ...  0  0  0]
 [ 6  1 12 ...  0  0  0]]
Labels: [1, 1]


## 3. Train our Models

### 3.1 BiLSTM Model - Baseline Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

import numpy as np

# Assuming X_padded is already defined as shown in previous steps
# Convert X_padded and y to NumPy arrays if they aren't already
X = np.array(X_padded)
y = np.array(y)

# Verify that X and y are now NumPy arrays
print(type(X), X.shape)
print(type(y), y.shape)


# Define the BiLSTM model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_length),
    Bidirectional(LSTM(units=50)),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
model.fit(X, y, epochs=5, batch_size=64, validation_split=0.2)  # Adjust epochs, batch_size, and validation_split as needed


<class 'numpy.ndarray'> (105190, 3999)
<class 'numpy.ndarray'> (105190,)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3999, 50)          6350      
                                                                 
 bidirectional_1 (Bidirecti  (None, 100)               40400     
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 46851 (183.01 KB)
Trainable params: 46851 (183.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
1315/1315 [==============================] - 290s 218ms/step - loss: 0.5329 - accuracy: 0.7354 - val_loss: 0.6213 - val_accuracy: 0

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
predictions = model.predict(test_X)
binary_predictions = (predictions > 0.5).astype(int)

# Evaluate the model
precision = precision_score(test_y, binary_predictions)
recall = recall_score(test_y, binary_predictions)
f1 = f1_score(test_y, binary_predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


79/79 [==============================] - 7s 77ms/step
Precision: 0.6819085487077535
Recall: 0.9510166358595195
F1 Score: 0.7942879197221151


### 3.2 DeBerta

In [ ]:


import sentencepiece
import torch
from torch.utils.data import Dataset
from transformers import DebertaV2Model, DebertaV2Config, DebertaV2Tokenizer, DebertaV2ForSequenceClassification
MODEL_NAME = 'microsoft/deberta-v3-base'
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=2)  # Adjust num_labels accordingly
config = DebertaV2Config.from_pretrained(MODEL_NAME)
tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Prepare dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=64)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
#tokenize dataset as per DeBerta
#split the data 80 and 20 first

train_dataset = TextDataset(all_sentences, y)
eval_dataset = TextDataset(test_all_sentences, test_y)
#! pip install --force-reinstall accelerate transformers[torch]
import torch, gc
gc.collect()
torch.cuda.empty_cache()



Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Training
# !pip install transformers[torch]
#!pip install accelerate -U


#! pip install --force-reinstall accelerate transformers[torch]

import torch, gc
gc.collect()
torch.cuda.empty_cache()

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


training_args = TrainingArguments(
    output_dir='./results',          # where to save the model
    evaluation_strategy="epoch",     # evaluate each epoch
    save_strategy="epoch",           # save model each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    num_train_epochs=3,              # number of epochs
    weight_decay=0.01,               # weight decay
    logging_dir='./logs',            # where to store logs
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # your encoded and prepared training dataset
    eval_dataset=eval_dataset,    # your encoded and prepared validation dataset
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],

)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.208600,0.249078,0.933147,0.874080,0.987985,0.927549
2,0.168200,0.193932,0.949560,0.903716,0.988909,0.944395
3,0.119500,0.263381,0.940753,0.886589,0.989834,0.935371


TrainOutput(global_step=39447, training_loss=0.1748655514895159, metrics={'train_runtime': 7520.6472, 'train_samples_per_second': 41.96, 'train_steps_per_second': 5.245, 'total_flos': 1.037893059825408e+16, 'train_loss': 0.1748655514895159, 'epoch': 3.0})

In [ ]:
model = DebertaV2ForSequenceClassification.from_pretrained('./results/checkpoint-39447')
test_trainer = Trainer(model)

raw_pred, _, _ = test_trainer.predict(eval_dataset)
y_pred = np.argmax(raw_pred, axis=1)
print(y_pred)
#tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)

#model.eval()


[1 1 1 ... 0 0 0]


In [ ]:
import pandas as pd
data = {
    'Sentence': test_all_sentences,  # Adjust field name as necessary
    'Predicted Label': y_pred,
    'Actual Label': test_y  # Adjust field name as necessary
}
df = pd.DataFrame(data)


In [ ]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
print(df_shuffled.head())


                                            Sentence  Predicted Label  \
0                      They are the best metal band.                1   
1  rapp because u can ryhmn and spit crapbut in r...                0   
2                   He didn't want to be a bigamist.                1   
3             i want a him and will call him 'money'                0   
4  Thus far, I have not heard that she returns he...                1   

   Actual Label  
0             1  
1             0  
2             0  
3             0  
4             1  


### 3.3 Bert (un-cased)

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback


import torch, gc
gc.collect()
torch.cuda.empty_cache()

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)



class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=64)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(all_sentences, y)
eval_dataset = TextDataset(test_all_sentences, test_y)
#! pip install --force-reinstall accelerate transformers[torch]



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

#! pip install --force-reinstall accelerate transformers[torch]

# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",     # evaluate each epoch
    save_strategy="epoch",           # save model each epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.518100,0.397858,0.871497,0.801745,0.934381,0.862996
2,0.572600,0.521684,0.784227,0.911988,0.555453,0.690408
3,0.694700,0.691717,0.566853,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=39447, training_loss=0.5511827216715427, metrics={'train_runtime': 4638.3315, 'train_samples_per_second': 68.035, 'train_steps_per_second': 8.505, 'total_flos': 1.03787444674944e+16, 'train_loss': 0.5511827216715427, 'epoch': 3.0})

In [ ]:
model_path = "./results/checkpoint-13149"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

raw_pred, _, _ = test_trainer.predict(eval_dataset)
y_pred = np.argmax(raw_pred, axis=1)
print(y_pred)



[1 0 1 ... 0 0 0]


In [ ]:
import pandas as pd
data = {
    'Sentence': test_all_sentences,  # Adjust field name as necessary
    'Predicted Label': y_pred,
    'Actual Label': test_y  # Adjust field name as necessary
}
df = pd.DataFrame(data)

df_shuffled = df.sample(frac=1).reset_index(drop=True)
print(df_shuffled.head())


                                            Sentence  Predicted Label  \
0  my favorite english song is kiss from a rose b...                1   
1                  neither...not a big fan of either                0   
2  I am not fond of any of them. Panget Rock Band...                1   
3           Goo Goo Dolls and Relient K are awesome!                0   
4              Cheesy I know...but I love this joke.                0   

   Actual Label  
0             0  
1             0  
2             1  
3             0  
4             0  


### 3.3 Simple CNN

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(all_sentences)

# Build model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 256

cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
cnn_model.add(Conv1D(64, 3, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)

cnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy", Precision(), Recall()])

In [ ]:

# Train model
X = np.array(X_padded)
y = np.array(y)
cnn_model.fit(X, y, batch_size=16, epochs=10, validation_split=0.2)

Epoch 1/10
5260/5260 [==============================] - 86s 16ms/step - loss: 0.4211 - accuracy: 0.8189 - precision_1: 0.8171 - recall_1: 0.9151 - val_loss: 0.4872 - val_accuracy: 0.7065 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/10
5260/5260 [==============================] - 83s 16ms/step - loss: 0.3787 - accuracy: 0.8386 - precision_1: 0.8384 - recall_1: 0.9188 - val_loss: 0.4072 - val_accuracy: 0.7948 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/10
5260/5260 [==============================] - 83s 16ms/step - loss: 0.3606 - accuracy: 0.8446 - precision_1: 0.8445 - recall_1: 0.9209 - val_loss: 0.5463 - val_accuracy: 0.7244 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 4/10
5260/5260 [==============================] - 83s 16ms/step - loss: 0.3504 - accuracy: 0.8488 - precision_1: 0.8490 - recall_1: 0.9221 - val_loss: 0.5037 - val_accuracy: 0.6914 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 5/10
5260/5260 [==

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
predictions = cnn_model.predict(test_X)
binary_predictions = (predictions > 0.5).astype(int)

# Evaluate the model
precision = precision_score(test_y, binary_predictions)
recall = recall_score(test_y, binary_predictions)
f1 = f1_score(test_y, binary_predictions)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


79/79 [==============================] - 1s 8ms/step
Precision: 0.7125256673511293
Recall: 0.9621072088724584
F1 Score: 0.818718049547778


In [ ]:
print(cnn_model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 3999, 256)         32512     
                                                                 
 conv1d_1 (Conv1D)           (None, 3997, 64)          49216     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                      

### Formality Score Calculations

In [ ]:
##Using the currently trained Bert Model, we can use it to predict formality score
model = BertForSequenceClassification.from_pretrained('./results/checkpoint-13149')
test_trainer = Trainer(model)

raw_pred, _, _ = test_trainer.predict(eval_dataset)
y_pred = np.argmax(raw_pred, axis=1)
print(y_pred)


[1 0 1 ... 0 0 0]


In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

test_50_sentences = df_shuffled["Sentence"].tolist()[:50]
test_50_labels = df_shuffled["Actual Label"].tolist()[:50]

In [ ]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
inputs = tokenizer(test_50_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
y_pred = y_pred[:50]
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    formality_scores = probabilities[:, 1].tolist()  # List of formality scores for each sentence


In [ ]:
for text, score, pred, model_prediction in zip(test_50_sentences, formality_scores, test_50_labels, y_pred):
    print(f"Text: {text}\nFormality Score: {score:.4f}\nOriginal Prediction: {pred}\nModel Prediction: {model_prediction}\n")

#1 for formal, 0 is informal --> the closer the score is to 0, the more informal it is, otherwise the closer it is to 1, then that is how formal it is.ok

Text: my favorite english song is kiss from a rose by seal.
Formality Score: 0.9577
Original Prediction: 0
Model Prediction: 1

Text: neither...not a big fan of either
Formality Score: 0.0453
Original Prediction: 0
Model Prediction: 0

Text: I am not fond of any of them. Panget Rock Band is the best.
Formality Score: 0.9579
Original Prediction: 1
Model Prediction: 1

Text: Goo Goo Dolls and Relient K are awesome!
Formality Score: 0.0453
Original Prediction: 0
Model Prediction: 1

Text: Cheesy I know...but I love this joke.
Formality Score: 0.0453
Original Prediction: 0
Model Prediction: 0

Text: You know the Beta team still wants to answer this.
Formality Score: 0.9589
Original Prediction: 1
Model Prediction: 1

Text: this does happen, like in batman, when he was recoving and that blond guy took over for a bit.
Formality Score: 0.3605
Original Prediction: 0
Model Prediction: 1

Text: A very close friend of mine is a Cap.
Formality Score: 0.9578
Original Prediction: 1
Model Prediction: 